# Import Hook

In [1]:
%load_ext literary.module

In [2]:
from nbconvert import Exporter
from traitlets import Instance, Type, default
from traitlets.config import Configurable

from ..transpile.exporter import LiteraryExporter
from .loader import NotebookLoader

Here we implement a `ProjectImporter` class. We expose the exporter class to later configuration

In [ ]:
class NotebookImporter(Configurable):
    exporter = Instance(Exporter)
    exporter_class = Type(LiteraryExporter).tag(config=True)

In [ ]:
@patch(NotebookImporter)
@default("exporter")
def _exporter_default(self):
    return self.exporter_class(parent=self)

In [6]:
@patch(NotebookImporter)
def get_loader(self, fullname, path):    
    # Create the exporter
    exporter = self.exporter_class(parent=self)
    return NotebookLoader(fullname, path, exporter=self.exporter)